In [45]:
import pandas as pd
import re

## Notebook with exploration of parsing VBA code for `CodePlan.py class VBAToCodePlan`
#### JDL, August 2023
</br></br>

### Regular Expression for Identifying and Parsing VBA Function/Sub First Lines

ChatGPT-Generated Regular Expression for function/sub first lines</br>
`Function ExampleProcedure(cls, ByVal arg1, Optional arg2) As Boolean`
</br></br>Initial Regular Expression and explanation:</br>
`r"Function\s+(\w+)\s*\((.*?)\)\s+As\s+(\w+)"`
</br></br>Modified with initial `?` make `As` clause optional:</br>
`r'^\s*(?:Function|Sub)\s+(\w+)\s*\((.*?)\)\s*(?:As\s+(\w+))?\s*$'`

1. `^\s*`: This part allows for optional leading whitespace at the beginning of the line.
2. `(?:Function|Sub)`: We use a non-capturing group (?: ... ) to match either "Function" or "Sub".
3. `\s+`: Allows for one or more whitespace characters after "Function" or "Sub".
4. `(\w+)`: This captures the function or sub name using the \w+ pattern, which matches one or more word characters (letters, digits, or underscores).
5. `\s*`: Allows for optional whitespace after the function or sub name.
6. `\((.*?)\)`: This captures the function arguments using the .*? pattern inside the parentheses, allowing for an empty argument list.
7. `\s*`: Allows for optional whitespace after the closing parenthesis of the argument list.
8. `(?:As\s+(\w+))?`: This is a non-capturing group that matches the "As" clause and the function type if present, but the whole group is optional due to the ? quantifier.
9. `\s*$`: This allows for optional trailing whitespace at the end of the line.

Notes:</br>
* Since the CodePlan.py strips leading and trailing whitespace on code lines, we don't need #1 and #9
* It's desirable to capture the first group to determine whether the line is for a Function or Subroutine
* Because of VBA auto-spacing, there will only be one whitespace between elements, so `\s` is ok instead of `\s+`
* General format is `(grp)\s(grp)\((grp)\)(?:\sAs\s(grp)?` where final `?` makes `As` clause optional and `\(` and `\)` are escaped parentheses

`r"(Function|Sub)\s(\w+)\((.*?)\)(?:\sAs\s(\w+))?"`

In [69]:
line1_pattern = r"(Function|Sub)\s(\w+)\((.*?)\)(?:\sAs\s(\w+))?"

def print_matches(line1_pattern, title, s):
    fn_match = re.match(line1_pattern, s)
    print('\n')
    print(title)
    print(True) if fn_match else print(False)
    if fn_match:
        for i in range(1,5):
            print(fn_match.group(i))

In [70]:
s = "Function ExampleProcedure(cls, ByVal arg1, Optional arg2) As Boolean"
print_matches(line1_pattern, "Example with match", s)

s = "Function ExampleProcedure() As Boolean"
print_matches(line1_pattern, "No Parentheses", s)

s = "Function ExampleProcedure(arg1)"
print_matches(line1_pattern, "No As Clause", s)

s = "Sub ExampleProcedure(arg1)"
print_matches(line1_pattern, "Sub", s)

s = "Sub   ExampleProcedure(arg1)"
print_matches(line1_pattern, "Non-Match", s)



Example with match
True
Function
ExampleProcedure
cls, ByVal arg1, Optional arg2
Boolean


No Parentheses
True
Function
ExampleProcedure

Boolean


No As Clause
True
Function
ExampleProcedure
arg1
None


Sub
True
Sub
ExampleProcedure
arg1
None


Non-Match
False


### Regular Expression for parsing Dim statements
Suggested by ChatGPT for this example:</br>
`"Dim var1 As Integer, var2 As String, var3 As New tbl"`</br></br>
` r"Dim\s+(?:(New\s+)?(\w+)\s+As\s+(\w+))(?:,\s+|$)"`</br>
* It needs the non-capturing group at the end to deal with optional commas after each variable's declaration
* the `/s+|$` matches either a whitespace character or (aka `|`) the end of the line represented by `$`
* `re.finditer` returns match[0] as overall string; populates 1, 2, 3 with possible groups including optional New
* `re.finditer` does not act as a Boolean. If non-match, nothing to iterate and `any(dim_match)` will be False (but then need to reset the iterator by repeating `re.finditer`)

In [149]:
dim_pattern = r"(\w+)\s(?:As\s)(New\s)?(\w+)(?:,\s*|$)"

def print_dim_matches(dim_pattern, title, s):
    dim_match = re.findall(dim_pattern, s)
    print('\n')
    print(title)
    print(dim_match)
        
s = "var1 As Integer"
print_dim_matches(dim_pattern, "single variable", s)

s = "var1 As Integer, var2 As String, var3 As New tbl"
print_dim_matches(dim_pattern, "3 Variables with New", s)

s = "var1   as Integer"
print_dim_matches(dim_pattern, "Non-match", s)



single variable
[('var1', '', 'Integer')]


3 Variables with New
[('var1', '', 'Integer'), ('var2', '', 'String'), ('var3', 'New ', 'tbl')]


Non-match
[]


### Refresher on interplay of index values (e.g. row "names") and row numbers

In [38]:
# Sample DataFrame with range index
data = {'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
        'Age': [25, 30, 22, 28, 35]}

df = pd.DataFrame(data)
print(df)

      Name  Age
0    Alice   25
1      Bob   30
2  Charlie   22
3    David   28
4      Eva   35


In [39]:
# Drop two rows to make index non-consecutive
df2 = df.drop(2).drop(0)
df2

,Name,Age
1,Bob,30
3,David,28
4,Eva,35


In [41]:
# index.get_loc returns the row number (e.g. .iloc row) for a specified index value
irow = df2.index.get_loc(3)
print(irow)
df2.iloc[irow]

1


Name    David
Age        28
Name: 3, dtype: object

In [43]:
#next row from a specified index
df2.iloc[df2.index.get_loc(3) + 1]

Name    Eva
Age      35
Name: 4, dtype: object

In [44]:
#2nd row's index value
df2.iloc[2].name

4

In [ ]:
### Parsing VBA Function/Sub Argument Strings

In [7]:
s = "arg1 As Integer, arg2, ByRef arg2, Optional ByVal arg3, Optional ByVal arg4 As Integer"

lst_args = s.split(", ")
lst_args

['arg1 As Integer',
 'arg2',
 'ByRef arg2',
 'Optional ByVal arg3',
 'Optional ByVal arg4 As Integer']

In [8]:
s2 = lst_args[4]
s2.split(" As ")

['Optional ByVal arg4', 'Integer']

In [9]:
def ParseArg(arg):
    HasType = False
    lst = arg.split(" As ")
    if len(lst) > 1:
        HasType = True
        lst = lst[0].split(" ") + [lst[1]]
    else:
        lst = arg.split(" ")
    return lst, HasType

In [10]:
for arg in lst_args:
    parsed_arg, HasType = ParseArg(arg)
    print('\n', parsed_arg, HasType)
    
    IsOptional, HasBy, arg_by = False, False, "ByRef"
    if parsed_arg[0] == "Optional":
        IsOptional = True
        parsed_arg = parsed_arg[1:]
    if parsed_arg[0] in ["ByRef", "ByVal"]: 
        HasBy = True
        arg_by = parsed_arg[0]
        parsed_arg = parsed_arg[1:]
    
    arg_name = parsed_arg[0]
    arg_type = "Variant"
    if HasType: arg_type = parsed_arg[-1]
    
    arg_code_plan = "|".join([arg_name, arg_type, arg_by])
    if IsOptional: arg_code_plan = arg_code_plan + "|Optional"
        
    #print(arg, "   ", arg_name, arg_type, arg_by, IsOptional)
    print(arg_code_plan)


 ['arg1', 'Integer'] True
arg1|Integer|ByRef

 ['arg2'] False
arg2|Variant|ByRef

 ['ByRef', 'arg2'] False
arg2|Variant|ByRef

 ['Optional', 'ByVal', 'arg3'] False
arg3|Variant|ByVal|Optional

 ['Optional', 'ByVal', 'arg4', 'Integer'] True
arg4|Integer|ByVal|Optional


### Docstring extraction

In [27]:
df_code = pd.DataFrame(index=range(10))
df_code["is_start"] = [False, False, False, True, False, False, False, False, True, False]
df_code["is_end"] = [False, False, False, False, False, True, False, False, False, True]
df_code["is_bound"] = [True, False, False, False, False, False, True, False, False, False]

df_code["code_stripped"] = ["\'------", "\' docstring1", "\' docstring2", 
              "Function First()", "Code1", "End", 
              "\'------", "\'docstring3", "Function Second()", "End"]

In [28]:
df_code

,is_start,is_end,is_bound,code_stripped
0,False,False,True,'------
1,False,False,False,' docstring1
2,False,False,False,' docstring2
3,True,False,False,Function First()
4,False,False,False,Code1
5,False,True,False,End
6,False,False,True,'------
7,False,False,False,'docstring3
8,True,False,False,Function Second()
9,False,True,False,End


In [32]:
fil_starts = df_code["is_start"]
fil_ends = df_code["is_end"]
fil_bounds = df_code["is_bound"]
df_code.drop(["is_start", "is_end"], inplace=True, axis=1)

In [35]:
def add_temp_docstring_col(df_code, fil_starts, fil_ends, fil_bounds):
    """
    Locate docstring rows between function/sub start rows and previous
    function/sub end row
    
    JDL 8/3/23
    """
    
    #Set previous end index for functions
    fil = fil_starts | fil_ends | fil_bounds
    df_code.loc[fil, 'prev_end_idx'] = df_code.loc[fil].index.to_series().shift(1, fill_value=0)
    
    # Parse docstrings for functions; +1 for first row after end except for idx_prev at file begin
    for idx in df_code[fil_starts].index:
        idx_prev = int(df_code.loc[idx, 'prev_end_idx'])
        if (idx_prev > 0) | (idx_prev in df_code[fil_bounds].index): idx_prev +=1

        #combine lines in block between previous end and function starts
        df_code.loc[idx, "docstring_temp"] = " \n".join(df_code.loc[list(range(idx_prev, idx)), "code_stripped"]) 
    return df_code[fil_starts]


In [36]:
add_temp_docstring_col(df_code, fil_starts, fil_ends, fil_bounds)

,is_bound,code_stripped,prev_end_idx,docstring_temp
3,False,Function First(),0.0,' docstring1 \n' docstring2
8,False,Function Second(),6.0,'docstring3


### Internal Variable Extraction

In [132]:
df_code = pd.DataFrame(index=range(10))
df_code["is_start"] = [False, True, False, False, False, False, True, False, False, False]
df_code["is_end"] =   [False, False, False, False, False, True, False, False, False, True]
df_code["is_dim"] =   [False, False, False, True, True, False, False, True, False, False]

df_code["code_stripped"] = ["\'------", "Function First()", "Something", 
              "Dim var1 As Integer, var2 As New Object", "Dim var3 As String", "End", 
              "Function Second()", "Dim var4 As New tbl", "Something", "End"]

In [133]:
fil_starts = df_code["is_start"]
fil_ends = df_code["is_end"]
fil_dims = df_code["is_dim"]
df_code.drop(["is_start", "is_end", "is_dim"], inplace=True, axis=1)

In [134]:
df_code.loc[fil_starts, "idx_start"] = df_code[fil_starts].index.values
df_code["idx_start"].ffill(inplace=True)
df_code

,code_stripped,idx_start
0,'------,NaN
1,Function First(),1.0
2,Something,1.0
3,"Dim var1 As Integer, var2 As New Object",1.0
4,Dim var3 As String,1.0
5,End,1.0
6,Function Second(),6.0
7,Dim var4 As New tbl,6.0
8,Something,6.0
9,End,6.0


In [204]:
dim_pattern = r"(\w+)\s(?:As\s)(New\s)?(\w+)(?:,\s*|$)"

def add_internal_var_col(df_code, fil_starts, fil_ends, fil_dims):
    """
    Locate VBA Dim rows and extract/parse variables into a lookup table
    by function/sub first line index
    
    JDL 8/3/23
    """
    df_lookup = df_code.copy()
    
    #Populate rows with index of their function/sub's first line
    df_lookup.loc[fil_starts, "idx_start"] = df_lookup[fil_starts].index.values
    df_lookup["idx_start"].ffill(inplace=True)
    
    #Initialize column for internal variable strings
    df_lookup["vars_internal"] = [[] for _ in range(len(df_lookup))]
    
    # Parse dim statements
    for idx in df_lookup[fil_dims].index:
        idx_start = int(df_lookup.loc[idx, "idx_start"])

        lst_parsed_dims = parse_dim_statement(dim_pattern, df_lookup.loc[idx, "code_stripped"])
                
        #Append the list items from current Dim statement onto any previous list items
        df_lookup.at[idx_start, "vars_internal"] = df_lookup.loc[idx_start, "vars_internal"] + lst_parsed_dims
    
    #Convert the list to string
    df_lookup = df_lookup[fil_starts]
    
    
    df_lookup["vars_internal"] = df_lookup["vars_internal"].apply(lambda lst: ",\n".join(map(str, lst)))
    #df['Column1'] = df['Column1'].apply(lambda lst: ",".join(map(str, lst)))
    return df_lookup

def parse_dim_statement(dim_pattern, s):
    
    #Pattern match on string without "Dim " prefix aka [4:] slice
    dim_match = re.findall(dim_pattern, s[4:])
    
    #Build list of parsed strings for each variable
    lst_parsed_vars = []
    for match in dim_match:
        
        #If match[1] populated, New descriptor is present
        lst_parsed = [match[0], match[2]]
        if len(match[1]) > 0: lst_parsed.append("New")
        s_parsed = "|".join(lst_parsed)
        
        #Append variable's parsed string to the list
        lst_parsed_vars.append(s_parsed)
    return lst_parsed_vars



In [205]:
s = "Dim var1 As Integer, var2 As New tbl"
parse_dim_statement(dim_pattern, s)

['var1|Integer', 'var2|tbl|New']

In [206]:
add_internal_var_col(df_code, fil_starts, fil_ends, fil_dims)

,code_stripped,idx_start,vars_internal
1,Function First(),1.0,"var1|Integer,\nvar2|Object|New,\nvar3|String"
6,Function Second(),6.0,var4|tbl|New
